# 텍스트 전처리

풀고자 하는 용도에 맞게 텍스트를 사전에 처리하는 작업이다. task에 맞게 전처리하지 않으면 제대로 동작하지 않는다.

## 1. 토큰화

#### 1) 단어 토큰화

토큰의 기준을 단어로 하는 경우로 단어 단위 외에도 의미를 갖는 문자열로 간주되기도 한다.( ex] 한국어의 어간, 문장 등)

Time is a illusion.을 토큰화 해보면 "Time", "is", "an", "illustion" 결과가 나온다. 이 경우는 토큰화 작업이 굉장히 간단하다. 구두점을 지우고 띄어쓰기를 기준으로 잘라낸다.

※ 구두점 : 마침표, 컴마, 물음표 등

하지만 보통 토큰화 작업은 단순히 구두점이나 특수문자를 전부 제거하는 정제 작업을 수행하는 것만으로 해결되지 않는다. 구두점이나 특수문자를 전부 제거하면 토큰이 의미를 잃어버리는 경우가 발생하기도 한다.

#### 2) 토큰화 중 생기는 선택의 순간

토큰화를 하다보면, 예상하지 못한 경우가 있어서 토큰화의 기준을 생각해봐야 하는 경우가 발생한다.(개발자의 직관이 필요한 부분)

ex) 영어권 언어의 '(어포스트로피)가 들어가 있는 단어는 어떻게 토큰으로 분류해야 하는지에 대한 선택의 문제

-> Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop.

다음 문장에서 Don't와 Jone's는 어떻게 토큰화할까? 토크나이저마다 다르게 토큰화하더라 word_tokenize는 'Do', "n't"로 분리하였다. 

반면 'Jone', "'s" 존은 이렇게 분리했다. WordPunctTokenizer는 구두점을 별도로 분류하는 특징을 갖고 있기 때문에 앞의 토크나이저와 달리 Don, ', t로 분리하였다. 

케라스의 text_to_word_sequence는 기본적으로 모든 알파벳을 소문자로 바꾸면서 구두점을 제거한다. 하지만 dont의 '인 아포스트로피는 보전하는 것을 알 수 있다.



In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# (Don't, Don t, Dont, Do n't 등) 다양한 선택지가 있다. 라이브러리 결과를 보자
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

```
word_tokenize는 'Do', "n't"로 분리하였다.

반면 'Jone', "'s" 존은 이렇게 분리했다.
```

In [3]:
print("word_tokenizer : ", word_tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop"))

word_tokenizer :  ['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


```
WordPunctTokenizer는 구두점을 별도로 분류하는 특징을 갖고 있기 때문에 앞의 토크나이저와 달리 Don, ', t로 분리하였다.
```

In [4]:
print("wordpunc_tokenizer : ", WordPunctTokenizer().tokenize("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop"))

wordpunc_tokenizer :  ['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


```
케라스의 text_to_word_sequence는 기본적으로 모든 알파벳을 소문자로 바꾸면서 구두점을 제거한다. 하지만 dont의 '인 아포스트로피는 보전하는 것을 알 수 있다.
```

In [5]:
print("text_to_word_sequence : ", text_to_word_sequence("Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop"))

text_to_word_sequence :  ["don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'mr', "jone's", 'orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']


#### 3) 토큰화에서 고려해야할 사항

토큰화 작업을 단순히 코퍼스에서 구두점을 제외하고 공백 기준으로 잘라내는 작업이라고 간주할 수 없다. 이보다 섬세한 알고리즘이 필요하다.

① 구두점이나 특수 문자를 단순 제외해서는 안 된다.

코퍼스에서 정제 작업을 진행하다보면 구두점조차도 하나의 토큰으로 분류하기도 한다. 가장 기본적인 예로 마침표 같은 경우는 문장의 경계를 알 수 있으므로 제외하지 않을 수 있다.

또한 단어 자체에 구두점을 갖고 있는 경우도 있는데 AT&T, Ph.D 등과 $45.55, 2022/10 등을 봐도 구두점이나 특수 문자를 단순 제외해서는 안된다.

<br>

② 줄임말과 단어 내에 띄어쓰기가 있는 경우
줄임말은 i'm은 i am으로 인식할 수 있어야하고, 단어 내 띄어쓰기는 rock 'n' roll을 보면 띄어쓰기가 있는 경우도 하나의 토큰으로 인식할 수 있어야한다/



③ 표준 토큰화 예제
표준으로 쓰이는 treebank의 규칙을 보자

규칙1 : -로 구성된 단어는 하나로 유지/ 규칙2 : doens't와 같이 '로 접어(i'm 이런거)가 함께하는단어는 분리



In [ ]:
# 트리뱅크 표준
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()
text = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."

print(tokenizer.tokenize(text))

['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


In [ ]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

text = "Starting a home-based restaurant may be an ideal. it doesn't have a food chain or restaurant of their own."
print('트리뱅크 워드토크나이저 :',tokenizer.tokenize(text))

트리뱅크 워드토크나이저 : ['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


#### 4) 문장 토큰화

토큰의 단위가 문장일 경우를 논의하자 

<br> 
갖고 있는 코퍼스 내에서 문장단위로 구분하는 작업으로 갖고 있는 코퍼스가 정제되지 않은 상태라면 문장 단위로 구분이 필요하다. 어떻게 할 수 있을까? 

<br> 
직관적으로는 마침표나 물음표로 문장을 자르면 되지 않을까?라고 생각할수 있지만 꼭 그런 건 아니다. !나 ?는 문장의 구분을 위한 명확한 구분자이지만 .온점은 그렇지 않다.

 "IP 111.111.111.111 서버에 들어가서 로그 파일을 저장해서 aaa@gmail.com으로 보내줘"라고 하면 이때 마침표를 기준으로 문장 토큰화를 하면 예상한 결과가 나오지 않을 수 있다.

★ 사용하는 코퍼스가 어떤 국적의 언어인지, 또는 해당 코퍼스 내에서 특수문자들이 어떻게 사용되고 있는지에 따라서 직접 규칙들을 정의해 볼수있겠다 -> 갖고 있는 코퍼스에 오타나 문장의 구성이 엉망이라면 정해 놓은 규칙이 소용 없을 수 있기 때문이다.



In [ ]:
# nltk의 sent_tokenize를 하면 문장 분리를 제공해준다. 

from nltk.tokenize import sent_tokenize

text = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to make sure no one was near."
print('문장 토큰화1 :',sent_tokenize(text))

문장 토큰화1 : ['His barber kept his word.', 'But keeping such a huge secret to himself was driving him crazy.', 'Finally, the barber went up a mountain and almost to the edge of a cliff.', 'He dug a hole in the midst of some reeds.', 'He looked about, to make sure no one was near.']


In [ ]:
# 중간 중간 온점이 등장하는 경우 -> nltk는 단순히 마침표를 구분자로 사용하지 않기 때문에 Ph.D.를 잘 인식한다.

text = "I am actively looking for Ph.D. students. and you are a Ph.D student."
print('문장 토큰화2 :',sent_tokenize(text))

문장 토큰화2 : ['I am actively looking for Ph.D. students.', 'and you are a Ph.D student.']


한국어의 경우 문장 토큰화 도구가 따로 존재한다.

kss(korean sentence splitter)이다.

In [6]:
# 한국어 문장 토큰화

!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
     |████████████████████████████████| 131 kB 47.5 MB/s 
  Created wheel for kss: filename=kss-3.7.3-py3-none-any.whl size=42449195 sha256=7adefb2a9c594d168b2145df83254ad7853bbc528b14683494f9a8935aa2f8f9
  Stored in directory: /root/.cache/pip/wheels/21/ae/be/1795119115db76f9824f02c419582c5c14dc4a6e8f144337a2
Successfully built kss


In [8]:
import kss

text = '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?'
print('한국어 문장 토큰화 :',kss.split_sentences(text))

한국어 문장 토큰화 : ['딥 러닝 자연어 처리가 재미있기는 합니다.', '그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다.', '이제 해보면 알걸요?']


#### 5) 한국어 토큰화의 어려움

영어는 New York 같은 합성어나 he's같은 준말에 대한 예외처리만 한다면, 띄어쓰기를 기준으로 토큰화를 수행해도 된다. 거의 대부분 단어 단위로 씌어쓰기가 이루어지기 때문에 띄어쓰기 토큰화와 단어 토큰화가 같다.

하지만 한국어는 영어와 달리 띄어쓰기만으로 하면 안된다.한국어의 경우에는 띄어쓰기 단위가 되는 단위를 어절이라고 하는데 어절 토큰화는 지양된다.

그 이유는 한국어가 교착어이기 때문이다. 교착어란 조사, 어미 등을 붙여서 말을 만드는 언어이다.

① 교착어의 특성

한국어는 영어와 달리 조사가 존재한다.그라는 단어 하나만 봐도 그가, 그에게, 그를 등 그라는 글자 뒤에 띄어쓰기 없이 바로 붙게된다.

같은 단어임에도 서로 다른 조사가 붙어서 다른 단어로 인식이 되면 자연어 처리가 힘들고 번거로워지는 경우가 많다. ∴ 대부분의 한국어 NLP에서 조사는 분리해줘야한다.

= 한국어는 어절이 독립적인 단어로 구성되는 것이 아니라 조사 등의 무언가가 붙어있는 경우가 많아서 이를 전부 분리해줘야 한다

★ 형태소 ★
한국어 토큰화에서는 형태소란 개념을 반드시 이해해야한다. 

형태소란? : 뜻을 가지는 가장 작은 말의 단위

=> 2가지 종류가 있다.

1. 자립 형태소 : 접사, 어미, 조사와 상관없이 자립하여 사용할 수 있는 형태소, 그 자체로 단어가 된다.

2. 의존 형태소 : 다른 형태소와 결합하여 사용되는 형태소. 접사, 어미, 조사, 어간를 말한다.

ex) 에디가 책을 읽었다.

이문장을 띄어쓰기 단위 토큰화를 한다면 ['에디가', '책을', '읽었다']의 결과이다.

형태소 단위로 분해하면 자립 형태소 : 에디, 책
의존 형태소 : -가, -을, 읽-, -었, -다

-> 한국어에서는 어절 토큰화가 아니라 형태소 토큰화를 수행해야한다.

② 한국어는 띄어쓰기가 영어보다 잘 지켜지지 않는다.

사용하는 한국어 코퍼스가 뉴스 기사와 같이 띄어쓰기를 철저하게 지키려고 노력하는 글이라면 좋겠지만, 많은 경우에 띄어쓰기가 틀렸거나 지켜지지 않는 코퍼스가 많다.

영어권 언어와 비교하여 띄어쓰기가 어렵고 잘 지켜지지 않는 경향이 있다. 

그 이유 : 한국어의 경우 띄어쓰기가 지켜지지 않아도 글을 쉽게 이해할 수 있는 언어이다.

영어의 경우 띄어쓰기를 하지 않으면 알아보기 어려운 문장들이 생긴다. 이는 언어적 특성의 차이이다.

#### 6) 품사 태깅

단어는 표기는 같지만 품사에 따라서 단어의 의미가 달라지기도 한다.fly는 동사로는 날다지만 명사로는 파리이다. 한국어에서도 못은 명사로는 고정하는 물건을 의미히자민 부사로서는 할 수 없다는 의미로 쓰인다.

결국 단어의 의미를 제대로 파악하기 위해서는 해당 단어가 어떤 품사로 쓰였는지 보는 것이 주요 지표가 될 수도 있다.

In [10]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

영어 문장에 대해서 토큰화를 수행한 결과를 입력으로 품사 태깅을 수행했다.

 PRP는 인칭 대명사, VBP는 동사, RB는 부사, VBG는 현재부사, IN은 전치사, NNP는 고유 명사, NNS는 복수형 명사, CC는 접속사, DT는 관사를 의미합니다.

In [11]:
# nltk에서는 treebank pos_tag라는 기준을 사용하여 품사를 태깅한다.
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

text = "I am actively looking for Ph.D. students. and you are a Ph.D. student."
tokenized_sentence = word_tokenize(text)

print('단어 토큰화 :',tokenized_sentence)
print('품사 태깅 :',pos_tag(tokenized_sentence))

단어 토큰화 : ['I', 'am', 'actively', 'looking', 'for', 'Ph.D.', 'students', '.', 'and', 'you', 'are', 'a', 'Ph.D.', 'student', '.']
품사 태깅 : [('I', 'PRP'), ('am', 'VBP'), ('actively', 'RB'), ('looking', 'VBG'), ('for', 'IN'), ('Ph.D.', 'NNP'), ('students', 'NNS'), ('.', '.'), ('and', 'CC'), ('you', 'PRP'), ('are', 'VBP'), ('a', 'DT'), ('Ph.D.', 'NNP'), ('student', 'NN'), ('.', '.')]


한국어 자연어 처리를 위해서는 KoNLPy라는 파이썬 패키지를 사용할 수 있다.

konlpy가 한국어 자연어 처리 패키지이고 여기서 다양한 기능을 제공한다.

그 중 형태소 분석기를 제공하는데 okt와 꼬꼬마를 해보자

In [13]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 46.1 MB/s 


okt 형태소 분석기로 토큰화를 시도해본 예제이다. 각각은 다음의 기능을 가지고 있다.


예제에서 형태소 추출과 품사 태깅 메소드의 결과를 보면 조사를 기본적으로 분리하고 있음을 확인할 수 있다.

In [14]:
# 형태소 분석기를 사용하여 토큰화를 진행

from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요")) 

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


각 형태소 분석기는 성능과 결과가 다르게 나오기 때문에, 형태소 분석기의 선택은 사용하고자 하는 필요 용도에 어떤 형태소 분석기가 가장 적절한지를 판단하고 사용해야한다.

In [15]:
print('꼬꼬마 형태소 분석 :',kkma.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('꼬꼬마 품사 태깅 :',kkma.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('꼬꼬마 명사 추출 :',kkma.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  

꼬꼬마 형태소 분석 : ['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가보', '아요']
꼬꼬마 품사 태깅 : [('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가보', 'VV'), ('아요', 'EFN')]
꼬꼬마 명사 추출 : ['코딩', '당신', '연휴', '여행']


## 정제 및 정규화

토큰화 작업 전, 후에는 텍스트 데이터를 용도에 맞게 정제(cleaning) 및 정규화(normalization)하는 일이 항상 함께한다.

=> 목적

정제(cleaning) : 갖고 있는 코퍼스로부터 노이즈 데이터를 제거한다.
정규화(normalization) : 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어준다.


정제 작업은 토큰화 작업에 방해가 되는 부분들을 배제시키고 토큰화 작업을 수행하기 위해서 토큰화 작업보다 앞서 이루어지기도 하지만, 토큰화 작업 이후에도 여전히 남아있는 노이즈들을 제거하기위해 지속적으로 이루어지기도 한다. 

사실 완벽한 정제 작업은 어려운 편이라서, 대부분의 경우 이 정도면 됐다.라는 일종의 합의점을 찾는다.

#### 1) 규칙 기반 표기가 다른 단어 통합

같은 의미를 갖고있음에도, 표기가 다른 단어들을 하나의 단어로 정규화하 는 방법을 사용한다.

USA와 US는 같은 의미를 가지므로 하나의 단어로 정규화해볼 수 있다. uh-huh와 uhhuh는 형태는 다르지만 여전히 같은 의미를 갖고 있다. 이에 어간추출, 표제어추출을 사용한다.

#### 2) 대, 소문자 통합

영어권 언어에서 대, 소문자를 통합하는 것은 단어의 개수를 줄일 수 있는 또 다른 정규화 방법이다.

소문자 변환이 왜 유용한지 예를 들어보자

페라리를 검색해본다고 하면, 엄밀히 말해서 사실 사용자가 검색을 통해 찾고자하는 결과는 a Ferrari car라고 봐야한다. 하지만 검색 엔진은 소문자 변환을 적용했을 것이기 때문에 ferrari만 쳐도 원하는 결과를 얻을 수 있다.

물론 무작정 전부다 소문자로 바꾸면 안 된다. 이런 규칙이 필요하다. 문장의 맨 앞에서 나오는 단어의 대문자만 소문자로 바꾸고, 다른 단어들은 전부 대문자인 상태로 유지



#### 3) 불필요한 단어 제거

노이즈 데이터(noise data)는 자연어가 아니면서 아무 의미도 갖지 않는 글자들(특수 문자 등)을 의미하기도 하지만, 분석하고자 하는 목적에 맞지 않는 불필요 단어들을 노이즈 데이터라고 한다.

1. 불용어 제거
2. 등장 빈도가 적은 단어

-> 때로는 텍스트 데이터에서 너무 적게 등장해서 자연어 처리에 도움이 되지 않는 단어들이 존재한다.100000개의 메일을 가지고 정상 메일에서는 어떤 단어들이 주로 등장하고, 스팸 메일에서는 어떤 단어들이 주로 등장하는지봐서 분류기를 만드려고 한다.

근데 100000개의 메일에서 총 5번 밖에 등장하지 않는 단어가 있다면 이 단어는 직관적으로 분류에 거의 도움이 되지 않은 것이다.



3. 길이가 짧은 단어

영어권 언어에서는 길이가 짧은 단어를 삭제하는 것만으로도 어느정도 자연어 처리에서 크게 의미가 없는 단어들을 제거하는 효과를 볼 수 있다

즉, 영어권 언어에서 길이가 짧은 단어들은 대부분 불용어에 해당한다.

하지만 한국어에서는 길이가 짧은 단어라고 삭제하는 이런 방법이 크게 유효하지 않을 수 있다.

영어권에서는 길이가 1인 단어를 제거하면 관사인 'a',나 주어인 'I'가 제거된다.

In [16]:
import re
text = "I was wondering if anyone out there could enlighten me on this car."

# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제
shortword = re.compile(r'\W*\b\w{1,2}\b')
print(shortword.sub('', text))

 was wondering anyone out there could enlighten this car.


## 3. 어간 추출과 표제어 추출

정규화 기법 중 코퍼스에 있는 단어의 개수를 줄일 수 있는 기법인 표제어 추출(lemmatization)과 어간 추출(stemming)

-> 서로 다른 단어들이지만, 하나의 단어로 일반화시킬 수 있다면 하나의 단어로 일반화시켜서 문서 내의 단어 수를 줄이겠다는 것

이러한 방법들은 단어의 빈도수를 기반으로 문제를 풀고자 하는 뒤에서 학습하게 될 BoW(Bag of Words) 표현을 사용하는 자연어 처리 문제에서 주로 사용된다.

★ 정규화의 지향점은 갖고 있는 코퍼스의 복잡성을 줄이는 일이다.

#### 1) 표제어 추출


표제어 추출은 단어들이 다른 형태를 가지더라도, 그 뿌리 단어를 찾아가서 단어의 개수를 줄일 수 있는지 판단한다.

예를 들어서 am, are, is는 서로 다른 스펠링이지만 그 뿌리 단어는 be라고 볼 수 있다. 이때, 이 단어들의 표제어는 be라고 한다.


형태소의 종류로 어간과 접사가 존재한다.

표제어 추출을 하는 가장 섬세한 방법은 이 두 가지 구성 요소를 분리하는 것이다. cats라면 어간 cat과 접사 s를 분리한다.fox는 더 이상 분리할 수 없다.



In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

표제어 추출은 어간 추출과는 달리 단어의 형태가 적절히 보존되는 양상을 보이는 특징이 있다.

하지만 dy나 ha와 같이 의미를 알 수 없는 적절하지 못한 단어를 출력하고 있다.

제어 추출기(lemmatizer)가 본래 단어의 품사 정보를 알아야만 정확한 결과를 얻을 수 있기 때문이다.

In [21]:
# NLTK에서는 표제어 추출을 위한 도구인 WordNetLemmatizer를 지원한다.

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']

print('표제어 추출 전 :',words)
print('표제어 추출 후 :',[lemmatizer.lemmatize(word) for word in words])

표제어 추출 전 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'lives', 'fly', 'dies', 'watched', 'has', 'starting']
표제어 추출 후 : ['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


In [22]:
lemmatizer.lemmatize('dies', 'v')

'die'

In [23]:
lemmatizer.lemmatize('has', 'v')

'have'

#### 2) 어간 추출

어간 추출은 형태학적 분석을 단순화한 버전이라고 볼 수도 있고, 정해진 규칙만 보고 단어의 어미를 자르는 어림짐작의 작업이라고 볼 수 있다.

이 작업은 섬세한 작업이 아니기 때문에 어간 추출 후에 나오는 결과 단어는 사전에 존재하지 않는 단어일 수도 있다.

규칙 기반의 접근을 하고 있으므로 어간 추출 후의 결과에는 사전에 없는 단어들도 포함된다.

규칙 기반의 접근한다.

In [24]:

from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

stemmer = PorterStemmer()

sentence = "This was not the map we found in Billy Bones's chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."
tokenized_sentence = word_tokenize(sentence)

print('어간 추출 전 :', tokenized_sentence)
print('어간 추출 후 :',[stemmer.stem(word) for word in tokenized_sentence])

어간 추출 전 : ['This', 'was', 'not', 'the', 'map', 'we', 'found', 'in', 'Billy', 'Bones', "'s", 'chest', ',', 'but', 'an', 'accurate', 'copy', ',', 'complete', 'in', 'all', 'things', '--', 'names', 'and', 'heights', 'and', 'soundings', '--', 'with', 'the', 'single', 'exception', 'of', 'the', 'red', 'crosses', 'and', 'the', 'written', 'notes', '.']
어간 추출 후 : ['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


In [25]:
# 상세 규칙은 마틴 포터의 홈페이지에서 확인 가능
words = ['formalize', 'allowance', 'electricical']

print('어간 추출 전 :',words)
print('어간 추출 후 :',[stemmer.stem(word) for word in words])

어간 추출 전 : ['formalize', 'allowance', 'electricical']
어간 추출 후 : ['formal', 'allow', 'electric']


이런 규칙에 기반한 알고리즘은 종종 제대로 된 일반화를 수행하지 못 할 수 있다. 

organization과 organ은 완전히 다른 단어 임에도 organization에 대해서 어간 추출을 했더니 organ이라는 단어가 나왔다. organ에 대해서 어간 추출을 한다고 하더라도 결과는 역시 organ이 되기 때문에, 두 단어에 대해서 어간 추출을 한다면 동일한 어간을 갖게 된다. 의미가 다른데 동일한 단어로 정규화되는 것은 정규화의 목적에 맞지 않다.



#### 3) 한국어에서의 어간 추출

한국어는 아래의 표와 같이 5언 9품사의 구조를 가지고 있다. 그중 용언은 어간과 어미의 결합으로 구성된다.

어간 : 용언(동사, 형용사)을 활용할 때, 원칙적으로 모양이 변하지 않는 부분

어미 : 용언의 어간 뒤에 붙어서 활용하면서 변하는 부분


1.활용 : 어간이 어미를 취할 때, 어간의모습이 일정하다면 규칙활용, 어간이나 어미의모습이 변하면 불규칙 활용이다.

ex) 긋다, 긋고, 그어서,그어라는 불규칙 활용이다.


2.규칙활용 : 규칙 활용은 어간이 어미를 취할 때, 어간의 모습이 일정하다. 아래의 예제는 어간과 어미가 합쳐질 때, 어간의 형태가 바뀌지 않는다.

규칙활용은 어간이 어미가 붙기전의 모습과 어미가 붙은 후의 모습이 같으므로, 규칙 기반으로 어미를 단순히 분리해주면 어간 추출이 된다.


3.불규칙 활용 : 불규칙 활용은 어간이 어미를 취할 때 어간의 모습이 바뀌거나 취하는 어미가 특수한 어미일 경우를 말한다.


예를 들어 ‘듣-, 돕-, 곱-, 잇-, 오르-, 노랗-’ 등이 ‘듣/들-, 돕/도우-, 곱/고우-, 잇/이-, 올/올-, 노랗/노라-’와 같이 어간의 형식이 달라지는 일이 있거나 ‘오르+ 아/어→올라, 하+아/어→하여, 이르+아/어→이르러, 푸르+아/어→푸르러’와 같이 일반적인 어미가 아닌 특수한 어미를 취하는 경우 불규칙 활용을 하는 예에 속한다.

이 경우에는 어간이 어미가 붙는 과정에서 어간의 모습이 바뀌었으므로 단순한 분리만으로 어간 추출이 되지 않고 좀 더 복잡한 규칙을 필요로 한다.



## 4. 불용어

갖고 있는 데이터에서 유의미한 단어 토큰만을 선별하기 위해서는 큰 의미가 없는 단어 토큰을 제거하는 작업이 필요하다.

여기서 큰 의미가 없다라는 것은 자주 등장하지만 분석을 하는 것에 있어서는 큰 도움이 되지 않는 단어들을 말한다.

예를 들면, I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있다.

이러한 단어들을 불용어라고 한다.

In [29]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

179개의 불용어를 사전에 정의하고 있고 10개 확인한 결과 i, me, my등을 불용어로 정의하고 있었다. 이는 문장에서 자주 등장하지만 실제 의미 분석을 하는데는 기여하는 바가 없는 것이겠다.

In [30]:
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어 10개 출력 :',stop_words_list[:10])

불용어 개수 : 179
불용어 10개 출력 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


제거 결과 is, not, an등이 제거 되었다.

In [33]:
# nltk로 불용어 제거하기

example = "Family is not an important thing. It's everything."
word_tokens = word_tokenize(example)

result = []

for word in word_tokens:
  if word not in stop_words_list:
    result.append(word)


print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)      

불용어 제거 전 : ['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
불용어 제거 후 : ['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


#### 한국어에서 불용어 제거하기

사용자가 직접 불용어 사전을 만들게 되는 경우가 많다.


보편적인 한국어 불용어 리스트 : https://www.ranks.nl/stopwords/korean

불용어가 많은 경우에는 코드 내에서 직접 정의하지 않고 txt 파일이나 csv 파일로 정리해놓고 이를 불러와서 사용하기도 한다.

In [34]:
okt = Okt()

example = "고기를 아무렇게나 구우려고 하면 안 돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
stop_words = "를 아무렇게나 구 우려 고 안 돼 같은 게 구울 때 는"

stop_words = stop_words.split(' ')
word_tokens = okt.morphs(example)

result = [word for word in word_tokens if not word in stop_words]

print('불용어 제거 전 :',word_tokens) 
print('불용어 제거 후 :',result)

불용어 제거 전 : ['고기', '를', '아무렇게나', '구', '우려', '고', '하면', '안', '돼', '.', '고기', '라고', '다', '같은', '게', '아니거든', '.', '예컨대', '삼겹살', '을', '구울', '때', '는', '중요한', '게', '있지', '.']
불용어 제거 후 : ['고기', '하면', '.', '고기', '라고', '다', '아니거든', '.', '예컨대', '삼겹살', '을', '중요한', '있지', '.']


## 5. 정규 표현식

텍스트 전처리에서 아주 유용한 도구로 특정 규칙이 있는 텍스트 데이터를 빠르게 정제할 수 있다. 정규 표현식은 문법과 메서드가 있다.

#### 실습

In [35]:
import re

.기호 : 한개의 문자를 의미

a.c라면 a와 c 사이에 어떤 1개의 문자라도 있는거면 규칙에 만족한다.


In [41]:
r = re.compile("a.c")
print(r.search("ac")) # 안됨
print(r.search("kkk")) # 안됨
print(r.search("abc")) # 규칙 만족하여 검색됨
print(r.search("43abcgfd")) # 문장이라면 규칙 만족하여 검색됨

None
None
<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(2, 5), match='abc'>


?기호 : ?앞에 없을 수도 있고 한개 있을 수도 있다.

.기호에서는 안된 abc와 ac 모두 매치된다.

In [43]:
r = re.compile("ab?c")
print(r.search("ac")) # 규칙 만족
print(r.search("abc")) # 규칙 만족
print(r.search("abbc")) # 두 개부터는 안됨

<re.Match object; span=(0, 2), match='ac'>
<re.Match object; span=(0, 3), match='abc'>
None


*기호 : *앞에 문자가 0개 이상이다.

ab*c라면 ac, abc와 ?에서 안된 abbc, abbbc도 만족한다.

In [46]:
r = re.compile("ab*c")
print(r.search("ac")) # 규칙 만족
print(r.search("abc")) # 규칙 만족
print(r.search("abbc")) # 규칙 만족
print(r.search("akbc")) # abc외에 다른 문자는 있으면 안된다.

<re.Match object; span=(0, 2), match='ac'>
<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(0, 4), match='abbc'>
None


+기호 : +앞에 문자가 1개 이상이다.

ab+c라면 *에서는된 ac는 만족하지 않는다.

In [47]:
r = re.compile("ab+c")
print(r.search("ac")) # 1개 미만은 안된다.
print(r.search("abc")) # 규칙 만족
print(r.search("abbc")) # 두 개부터는 안됨
print(r.search("akbc")) # abc외에 다른 문자는 있으면 안된다.

None
<re.Match object; span=(0, 3), match='abc'>
<re.Match object; span=(0, 4), match='abbc'>
None


^기호 : ^로 시작되는 문자열

In [48]:
r = re.compile("^ab")
print(r.search("ab")) # 규칙 만족
print(r.search("abbc")) # 규칙 만족
print(r.search("kkk")) # ab로 시작하지 않는다.
print(r.search("bbc")) # ab로 시작하지 않는다.

<re.Match object; span=(0, 2), match='ab'>
<re.Match object; span=(0, 2), match='ab'>
None
None


{숫자}기호 : 해당 문자를 숫자만큼 반복한다.

ab{2}a라면 a와 c사이에 b가 존재하면서 b가 2개인 문자열일 경우 만족한다.

In [50]:
r = re.compile("ab{2}c")
print(r.search("ac")) # b가 2개있어야한다.
print(r.search("abc")) # b가 2개있어야한다.
print(r.search("abbbbc")) # b가 2개있어야한다.
print(r.search("abbc")) # 규칙 만족
print(r.search("abkkc")) # a, c사이에 다른 문자는 있으면 안된다.

None
None
None
<re.Match object; span=(0, 4), match='abbc'>
None


{숫자1, 숫자2}기호 : 해당 문자를 숫자1 이상 숫자2 이하만큼 반복한다.

ab{2, 8}a라면 a와 c사이에 b가 존재하면서 b가 2개 이상 8개 이하인 문자열일 경우 만족한다.

-> ,에 띄어쓰기가 있으면 안된다.

In [52]:
r = re.compile("ab{2,8}c")
print(r.search("ac")) # b가 2개있어야한다.
print(r.search("abc")) # b가 2개있어야한다.
print(r.search("abbbbc")) # b가 2개있어야한다.
print(r.search("abbc")) # 규칙 만족
print(r.search("abkkc")) # a, c사이에 다른 문자는 있으면 안된다.

None
None
<re.Match object; span=(0, 6), match='abbbbc'>
<re.Match object; span=(0, 4), match='abbc'>
None


[]기호 : []안에 문자들을 넣으면 그 문자들 중 한 개의 문자와 매치라는 의미를 가진다.

[abc]라면 a또는 b또는 c가 들어가 있는 문자열과 매치된다.

-> 범위 지정
[a-zA-Z]는 알파벳 전부를 의미하며 [0-9]는 숫자 전부를 의미

-> 결과는 정규표현식에 속하는 문자 하나 각각을 리턴 > {2}를 사용하여 문자 한개 이상으로 가능

In [75]:
r = re.compile("[abc]")
print(r.search("a")) # 규칙 만족
print(r.search("vsl")) # a 또는 b 또는 c가 없다.
print(r.search("kak")) # 규칙 만족
print(r.search("bbc")) # 규칙 만족
print(r.search("bbc vcb mxv")) # 규칙 만족
print(r.findall("bbc vcb mxv")) # 규칙 만족

<re.Match object; span=(0, 1), match='a'>
None
<re.Match object; span=(1, 2), match='a'>
<re.Match object; span=(0, 1), match='b'>
<re.Match object; span=(0, 1), match='b'>
['b', 'b', 'c', 'c', 'b']


In [56]:
# 알파벳 소문자에 대한 범위

r = re.compile("[a-z]")

print(r.search("AAA")) # 아무런 결과도 출력되지 않는다.
print(r.search("111") ) # 아무런 결과도 출력되지 않는다.
print(r.search("a434") ) # 조건 만족

None
None
<re.Match object; span=(0, 1), match='a'>


[^문자]기호 : ^기호 뒤에 붙은 문자들을 제외한 모든 문자인 경우 만족

In [57]:
r = re.compile("[^abc]")
print(r.search("a")) # a가 존재
print(r.search("abbc")) # a 또는 b 또는 c가 존재
print(r.search("kkk")) # 규칙 만족
print(r.search("bbc")) # b가 존재

None
None
<re.Match object; span=(0, 1), match='k'>
None


#### 모듈 함수 실습

#### 1) re.match

search는 문자열 전체에서 매치하는지 본다면 match는 첫 부분을 본다. 문자열의 시작에서 패턴이 일치하지 않으면 찾지 않는다.

In [58]:
r = re.compile("ab.")
r.match("kkkabc") # 아무런 결과도 출력되지 않는다.

In [59]:
r.search("kkkabc")  

<re.Match object; span=(3, 6), match='abc'>

In [60]:
r.match("abckkk") # 첫 부분이 일치한다면 찾는다.

<re.Match object; span=(0, 3), match='abc'>

#### 2) re.split()

split() 함수는 입력된 정규 표현식을 기준으로 문자열들을 분리하여 리스트로 리턴한다.

In [61]:
# 공백 기준 분리
text = "사과 딸기 수박 메론 바나나"
re.split(" ", text)

['사과', '딸기', '수박', '메론', '바나나']

In [62]:
# 줄바꿈 기준 분리
text = """사과
딸기
수박
메론
바나나"""

re.split("\n", text)

['사과', '딸기', '수박', '메론', '바나나']

#### 3) re.findall()

search는 정규표현식과 매치되는 문자열 하나만 리턴했는데 이번에는 모든 문자열을 리스트로 리턴한다.

-> 결과 : 역시 []에 속하는 문자 각각을 리턴

In [64]:
# 한글 정규표현식 
text = """이름 : 김철수
전화번호 : 010 - 1234 - 1234
나이 : 30
성별 : 남"""

re.findall("[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]", text)

['이', '름', '김', '철', '수', '전', '화', '번', '호', '나', '이', '성', '별', '남']

In [ ]:
# 숫자 정규표현식 \d
re.findall("\d+", text)

#### 4) re.sub()

sub() 함수는 정규 표현식 패턴과 일치하는 문자열을 찾아 다른 문자열로 대체할 수 있다.

알파벳을 제외한 특수문자를 공백으로 처리하고 싶다면

In [65]:
# [^문자]기호 : ^기호 뒤에 붙은 문자들을 제외한 모든 문자인 경우 만족

text = "Regular expression : A regular expression, regex or regexp[1] (sometimes called a rational expression)[2][3] is, in theoretical computer science and formal language theory, a sequence of characters that define a search pattern."

preprocessed_text = re.sub('[^a-zA-Z]', ' ', text)
print(preprocessed_text)

Regular expression   A regular expression  regex or regexp     sometimes called a rational expression        is  in theoretical computer science and formal language theory  a sequence of characters that define a search pattern 


#### 정규 표현식 텍스트 전처리 예제

In [67]:
# {숫자}기호 : 해당 문자를 숫자만큼 반복한다.

text = """100 John    PROF
101 James   STUD
102 Mac   STUD"""

re.findall('[A-Z]{4}',text) # 대문자가 연속으로 번 등장하는 경우

['PROF', 'STUD', 'STUD']

In [68]:
# 정규 표현식을 이용한 토큰화
# -> 사용자 지정 규칙으로 토큰화를 할 수 있다.

# 토크나이저 1은 \w+는 문자 또는 숫자가 1개 이상인 경우를 의미한다.
# 토크나이저 2는 공백을 기준으로 토큰화


from nltk.tokenize import RegexpTokenizer

text = "Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop"

tokenizer1 = RegexpTokenizer("[\w]+")
tokenizer2 = RegexpTokenizer("\s+", gaps=True)

print(tokenizer1.tokenize(text))
print(tokenizer2.tokenize(text))

['Don', 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'Mr', 'Jone', 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']
["Don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name,', 'Mr.', "Jone's", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']
